<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width = 300, align = "center"></a>

<h1 align=center><font size = 5>Lab: Working with a real world data-set using SQL and Python</font></h1>

# Introduction

This notebook shows how to work with a real world dataset using SQL and Python. In this lab you will:
1. Understand the dataset for Chicago Public School level performance 
1. Store the dataset in an Db2 database on IBM Cloud instance
1. Retrieve metadata about tables and columns and query data from mixed case columns
1. Solve example problems to practice your SQL skills including using built-in database functions

## Chicago Public Schools - Progress Report Cards (2011-2012) 

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true

__NOTE__: Do not download the dataset directly from City of Chicago portal. Instead download a more database friendly version from the link below.
Now download a static copy of this database and review some of its contents:
https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv



### Store the dataset in a Table
In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, __it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II__. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called __SCHOOLS__.

<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>

### Connect to the database
Let us now load the ipython-sql  extension and establish a connection with the database

In [18]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [19]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
%sql ibm_db_sa://mjs55547:1c1z3flj%4099dq0g3@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB

'Connected: mjs55547@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created

In [20]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
MJS55547,SCHOOLS,MJS55547,U,T,N,None,None,None,None,2019-08-08 14:14:51.702696,2019-08-08 14:14:51.702696,2019-08-08 14:14:51.702696,None,78,13,2357,-1,-1,-1,-1,-1,-1,-1,-1,mjs55547space1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,-1,N,999,-1,-1.0,-1,-1.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,-1,2019-08-08 14:14:51.702696,0,,None,None,N,MJS55547,,,,0001-01-01,,N,R,N,-1.0,None


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.TABLES contains the table metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='YOUR-DB2-USERNAME'

or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      
or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'SCHOOLS'

-->

### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?

In [21]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from SYSCAT.COLUMNS where TABNAME='SCHOOLS'

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
78


Double-click __here__ for a hint

<!--
In Db2 the system catalog table called SYSCAT.COLUMNS contains the column metadata
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

-->

Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.

In [22]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
School_ID,INTEGER,4
NAME_OF_SCHOOL,VARCHAR,65
"Elementary, Middle, or High School",VARCHAR,2
Street_Address,VARCHAR,30
City,VARCHAR,7
State,VARCHAR,2
ZIP_Code,INTEGER,4
Phone_Number,VARCHAR,14
Link,VARCHAR,78
Network_Manager,VARCHAR,40


Double-click __here__ for the solution.

<!-- Solution:

%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SCHOOLS'

or

%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'SCHOOLS'

-->

### Questions
1. Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
1. What is the name of "Community Area Name" column in your table? Does it have spaces?
1. Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "_"?

## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?

In [23]:
%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
462


Double-click __here__ for a hint

<!--
Which column specifies the school type e.g. 'ES', 'MS', 'HS'?
-->

Double-click __here__ for another hint

<!--
Does the column name have mixed case, spaces or other special characters?
If so, ensure you use double quotes around the "Name of the Column"
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select count(*) from SCHOOLS where "Elementary, Middle, or High School" = 'ES'

Correct answer: 462

-->

### Problem 2

##### What is the highest Safety Score?

In [24]:
%sql SELECT MAX(SAFETY_SCORE) FROM SCHOOLS

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


1
99


Double-click __here__ for a hint

<!--
Use the MAX() function
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql select MAX(Safety_Score) AS MAX_SAFETY_SCORE from SCHOOLS

Correct answer: 99
-->


### Problem 3

##### Which schools have highest Safety Score?

In [25]:
%sql SELECT * FROM SCHOOLS WHERE SAFETY_SCORE = (SELECT MAX(SAFETY_SCORE) FROM SCHOOLS)

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,healthy_school_certified,Safety_Icon,safety_score,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,average_student_attendance,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,college_enrollment,General_Services_Route,Freshman_on_Track_Rate__,x_coordinate,y_coordinate,Latitude,Longitude,community_area_number,community_area_name,Ward,Police_District,Location
610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago,IL,60614,(773) 534-5720,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610038.pdf,Fullerton Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,Yes,Very Strong,99,Very Strong,99,Strong,74,Strong,66,Strong,65,Strong,70,Strong,56,Average,47,96.00%,2.0,96.40%,95.80%,80.1,43.3,89.6,84.9,60.7,62.6,81.9,85.2,52,62.4,66.3,77.9,69.7,64.4,0.2,0.9,Yellow,Green,67.1,54.5,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,813,33,NDA,1171699.458,1915829.428,41.92449696,-87.64452163,7,LINCOLN PARK,43,18,"(41.92449696, -87.64452163)"
609799,Alexander Graham Bell Elementary School,ES,3730 N Oakley Ave,Chicago,IL,60618,(773) 534-5150,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609799.pdf,Ravenswood-Ridge Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,No,Very Strong,99,Very Strong,88,Strong,64,Average,46,Average,51,Average,51,NDA,NDA,NDA,NDA,96.30%,6.3,95.90%,99.30%,91.9,67.3,79.2,77.4,53.3,54.7,84.2,83,49.8,53.6,62.6,71.7,64.0,57.9,0.0,0.3,Yellow,Yellow,58.1,65.6,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,998,35,NDA,1160327.881,1924862.722,41.94952795,-87.68605496,5,NORTH CENTER,47,19,"(41.94952795, -87.68605496)"
610084,Annie Keller Elementary Gifted Magnet School,ES,3020 W 108th St,Chicago,IL,60655,(773) 535-2636,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610084.pdf,Rock Island Elementary Network,FAR SOUTH SIDE COLLABORATIVE,Yes,Standard,Not on Probation,Level 1,No,Very Strong,99,Very Strong,97,Very Strong,85,Very Strong,82,Very Strong,94,Very Strong,82,Strong,68,Strong,60,97.50%,4.9,96.50%,100.00%,100,NDA,100,100,63.4,74.7,96.5,97.7,61.2,76.7,89.3,100,92.8,92.3,1.6,2.3,Green,Green,51.5,70.6,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,245,49,NDA,1157959.455,1832892.067,41.69719792,-87.69726380,74,MOUNT GREENWOOD,19,22,"(41.69719792, -87.6972638)"
609820,Augustus H Burley Elementary School,ES,1630 W Barry Ave,Chicago,IL,60657,(773) 534-5475,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609820.pdf,Ravenswood-Ridge Elementary Network,NORTH-NORTHWEST SIDE COLLABORATIVE,Yes,Standard,Not on Probation,Level 1,No,Very Strong,99,NDA,NDA,Strong,78,Strong,65,NDA,NDA,NDA,NDA,Strong,59,Average,49,96.50%,0.7,95.00%,97.90%,69.4,47,64.5,70.2,51.2,66.7,69.4,75,62.9,67.3

Double-click __here__ for the solution.

<!-- Solution:
In the previous problem we found out that the highest Safety Score is 99, so we can use that as an input in the where clause:

%sql select Name_of_School, Safety_Score from SCHOOLS where Safety_Score = 99

or, a better way:

%sql select Name_of_School, Safety_Score from SCHOOLS where \
  Safety_Score= (select MAX(Safety_Score) from SCHOOLS)


Correct answer: several schools with with Safety Score of 99.
-->


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [26]:
%sql SELECT * FROM SCHOOLS ORDER BY Average_Student_Attendance desc LIMIT 10

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,healthy_school_certified,Safety_Icon,safety_score,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,average_student_attendance,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,college_enrollment,General_Services_Route,Freshman_on_Track_Rate__,x_coordinate,y_coordinate,Latitude,Longitude,community_area_number,community_area_name,Ward,Police_District,Location
610504,Velma F Thomas Early Childhood Center,ES,3625 S Hoyne Ave,Chicago,IL,60609,(773) 535-4088,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610504.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,NDA,Standard,NDA,NDA,No,NDA,None,NDA,NDA,NDA,None,NDA,None,NDA,NDA,NDA,NDA,Strong,55,Strong,57,None,0.0,95.60%,100.00%,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,None,None,None,None,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,211,39,NDA,1162992.823,1880511.570,41.82776914,-87.67750501,59,MCKINLEY PARK,11,9,"(41.82776914, -87.67750501)"
609959,John Charles Haines Elementary School,ES,247 W 23rd Pl,Chicago,IL,60616,(773) 534-9200,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609959.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,No,Standard,Not on Probation,Level 1,No,Weak,32,Average,47,Weak,37,Weak,37,Average,42,Average,48,Weak,43,Weak,43,98.40%,0.3,96.20%,96.90%,70.2,47.6,69.3,57.6,69,69.8,73.2,53.6,74.3,63.6,35.9,32,45.0,25.7,1.0,0.5,Green,Yellow,25.9,93.1,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,664,40,NDA,1174842.529,1888681.328,41.84993116,-87.63378596,34,ARMOUR SQUARE,25,9,"(41.84993116, -87.63378596)"
610217,James Ward Elementary School,ES,2701 S Shields Ave,Chicago,IL,60616,(773) 534-9050,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610217.pdf,Pershing Elementary Network,SOUTHWEST SIDE COLLABORATIVE,Yes,Standard,Not on Probation,Level 1,No,Average,49,Strong,65,Average,50,Average,46,Strong,63,Average,56,Average,48,Weak,43,97.80%,1.3,96.90%,100.00%,74.8,63.8,71.6,57,68.1,80.6,77.8,53.3,79.3,51.9,44.2,34.6,51.1,23.8,-0.1,-1.0,Yellow,Red,43.6,50,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,492,40,NDA,1174442.789,1886499.005,41.84395162,-87.63531815,34,ARMOUR SQUARE,11,9,"(41.84395162, -87.63531815)"
610132,Edgar Allan Poe Elementary Classical School,ES,10538 S Langley Ave,Chicago,IL,60628,(773) 535-5525,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_610132.pdf,Lake Calumet Elementary Network,FAR SOUTH SIDE COLLABORATIVE,Yes,Standard,Not on Probation,Level 1,No,Very Strong,99,Weak,33,Strong,66,Very Strong,88,Weak,27,Average,41,Strong,55,Average,47,97.60%,0.0,97.20%,100.00%,100,78.3,94,91.6,51.8,65.5,78.6,92.9,53.6,60.7,NDA,NDA,78.8,79.6,1.7,-0.2,Green,Yellow,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,192,48,NDA,1182742.505,1835063.900,

Double-click __here__ for the solution.

<!-- Solution:

%sql select Name_of_School, Average_Student_Attendance from SCHOOLS \
    order by Average_Student_Attendance desc nulls last limit 10 

-->

### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance

In [27]:
%sql SELECT * FROM SCHOOLS ORDER BY Average_Student_Attendance FETCH FIRST 5 ROWS ONLY

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,"Elementary, Middle, or High School",Street_Address,City,State,ZIP_Code,Phone_Number,Link,Network_Manager,Collaborative_Name,Adequate_Yearly_Progress_Made_,Track_Schedule,CPS_Performance_Policy_Status,CPS_Performance_Policy_Level,healthy_school_certified,Safety_Icon,safety_score,Family_Involvement_Icon,Family_Involvement_Score,Environment_Icon,Environment_Score,Instruction_Icon,Instruction_Score,Leaders_Icon,Leaders_Score,Teachers_Icon,Teachers_Score,Parent_Engagement_Icon,Parent_Engagement_Score,Parent_Environment_Icon,Parent_Environment_Score,average_student_attendance,Rate_of_Misconducts__per_100_students_,Average_Teacher_Attendance,Individualized_Education_Program_Compliance_Rate,Pk_2_Literacy__,Pk_2_Math__,Gr3_5_Grade_Level_Math__,Gr3_5_Grade_Level_Read__,Gr3_5_Keep_Pace_Read__,Gr3_5_Keep_Pace_Math__,Gr6_8_Grade_Level_Math__,Gr6_8_Grade_Level_Read__,Gr6_8_Keep_Pace_Math_,Gr6_8_Keep_Pace_Read__,Gr_8_Explore_Math__,Gr_8_Explore_Read__,ISAT_Exceeding_Math__,ISAT_Exceeding_Reading__,ISAT_Value_Add_Math,ISAT_Value_Add_Read,ISAT_Value_Add_Color_Math,ISAT_Value_Add_Color_Read,Students_Taking__Algebra__,Students_Passing__Algebra__,9th Grade EXPLORE (2009),9th Grade EXPLORE (2010),10th Grade PLAN (2009),10th Grade PLAN (2010),Net_Change_EXPLORE_and_PLAN,11th Grade Average ACT (2011),Net_Change_PLAN_and_ACT,College_Eligibility__,Graduation_Rate__,College_Enrollment_Rate__,college_enrollment,General_Services_Route,Freshman_on_Track_Rate__,x_coordinate,y_coordinate,Latitude,Longitude,community_area_number,community_area_name,Ward,Police_District,Location
609702,Richard T Crane Technical Preparatory High School,HS,2245 W Jackson Blvd,Chicago,IL,60612,(773) 534-7550,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609702.pdf,West Side High School Network,WEST SIDE COLLABORATIVE,No,Standard,Probation,Level 3,No,Average,43,NDA,NDA,Average,50,Average,48,NDA,NDA,NDA,NDA,Average,47,Weak,46,57.90%,19.9,93.90%,99.10%,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,None,None,None,None,NDA,NDA,NDA,NDA,12,11.9,13.2,13.6,1.6,14,0.8,3.7,48,32.6,478,38,40.9,1161290.138,1898611.918,41.87747384,-87.68324922,28,NEAR WEST SIDE,2,12,"(41.87747384, -87.68324922)"
609871,Barbara Vick Early Childhood & Family Center,ES,2554 W 113th St,Chicago,IL,60655,(773) 535-2671,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609871.pdf,Rock Island Elementary Network,FAR SOUTH SIDE COLLABORATIVE,NDA,Track_E,NDA,NDA,No,NDA,None,NDA,NDA,NDA,None,NDA,None,NDA,NDA,NDA,NDA,Strong,66,Strong,60,60.90%,0.0,95.50%,100.00%,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,None,None,None,None,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,204,49,NDA,1161152.120,1829643.669,41.68821843,-87.68566340,75,MORGAN PARK,19,22,"(41.68821843, -87.6856634)"
609736,Dyett High School,HS,555 E 51st St,Chicago,IL,60615,(773) 535-1825,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609736.pdf,South Side High School Network,SOUTH SIDE COLLABORATIVE,No,Track_E,Probation,Level 3,No,Weak,27,Very Weak,18,Weak,35,Average,47,Weak,35,Weak,25,NDA,NDA,NDA,NDA,62.50%,24.4,93.50%,100.00%,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,None,None,None,None,NDA,NDA,NDA,NDA,11.6,11.7,13,12.9,1.3,14,1,3.3,33.7,51,318,42,37.1,1180944.201,1871282.832,41.80204982,-87.61192836,40,WASHINGTON PARK,4,2,"(41.80204982, -87.61192836)"
609727,Wendell Phillips Academy High School,HS,244 E Pershing Rd,Chicago,IL,60653,(773) 535-1603,http://schoolreports.cps.edu/SchoolProgressReport_Eng/Spring2011Eng_609727.pdf,AUSL Schools,SOUTH SIDE COLLABORATIVE,No,Track_E,Probation,Level 3,No,NDA,None,Strong,71,NDA,None,NDA,None,Very Strong,80,Strong,79,NDA,NDA,NDA,NDA,63.00%,22.0,96.10%,100.00%,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,NDA,None,None,None,None,NDA,NDA,NDA,NDA,11.5,12.8,13.4,12.9,1.4,15,1.6,2.5,40.3,42.6,590,40,32.1,1178735.106,1879229.780,41.82390751,-87.61978794,35,DOUGLAS,3,2,"(41.82390751, -87.61978794)"
610389,Orr Academy High

Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column

In [35]:
%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', ' ') as ASA FROM SCHOOLS ORDER BY Average_Student_Attendance FETCH FIRST 5 ROWS ONLY

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,asa
Richard T Crane Technical Preparatory High School,57.90
Barbara Vick Early Childhood & Family Center,60.90
Dyett High School,62.50
Wendell Phillips Academy High School,63.00
Orr Academy High School,66.30


Double-click __here__ for a hint

<!--
Use the REPLACE() function to replace '%' with ''
See documentation for this function at:
https://www.ibm.com/support/knowledgecenter/en/SSEPGG_10.5.0/com.ibm.db2.luw.sql.ref.doc/doc/r0000843.html
-->

Double-click __here__ for the solution.

<!-- Hint:

%sql SELECT Name_of_School, REPLACE(Average_Student_Attendance, '%', '') \
     from SCHOOLS \
     order by Average_Student_Attendance \
     fetch first 5 rows only

-->


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?

In [46]:
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Barbara Vick Early Childhood & Family Center,60.90%
Chicago Vocational Career Academy High School,68.80%
Dyett High School,62.50%
Manley Career Academy High School,66.80%
Orr Academy High School,66.30%
Richard T Crane Technical Preparatory High School,57.90%
Roberto Clemente Community Academy High School,69.60%
Wendell Phillips Academy High School,63.00%


Double-click __here__ for a hint

<!--
The datatype of the "Average_Student_Attendance" column is varchar.
So you cannot use it as is in the where clause for a numeric comparison.
First use the CAST() function to cast it as a DECIMAL or DOUBLE
e.g. CAST("Column_Name" as DOUBLE)
or simply: DECIMAL("Column_Name")
-->

Double-click __here__ for another hint

<!--
Don't forget the '%' age sign needs to be removed before casting
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where CAST ( REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     
or,

%sql SELECT Name_of_School, Average_Student_Attendance  \
     from SCHOOLS \
     where DECIMAL ( REPLACE(Average_Student_Attendance, '%', '') ) < 70 \
     order by Average_Student_Attendance

-->


### Problem 8

##### Get the total College Enrollment for each Community Area

In [48]:
%sql SELECT Community_Area_Name, SUM(College_Enrollment) AS TOTAL_ENROLLMENT FROM SCHOOLS GROUP BY Community_Area_Name

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,total_enrollment
ALBANY PARK,6864
ARCHER HEIGHTS,4823
ARMOUR SQUARE,1458
ASHBURN,6483
AUBURN GRESHAM,4175
AUSTIN,10933
AVALON PARK,1522
AVONDALE,3640
BELMONT CRAGIN,14386
BEVERLY,1636


Double-click __here__ for a hint

<!--
Verify the exact name of the Enrollment column in the database
Use the SUM() function to add up the Enrollments for each Community Area
-->

Double-click __here__ for another hint

<!--
Don't forget to group by the Community Area
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name 

-->


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order 

In [51]:
%sql SELECT Community_Area_Name, SUM(College_Enrollment) AS TOTAL_ENROLLMENT FROM SCHOOLS GROUP BY Community_Area_Name ORDER BY TOTAL_ENROLLMENT asc FETCH FIRST 5 ROWS ONLY

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,total_enrollment
OAKLAND,140
FULLER PARK,531
BURNSIDE,549
OHARE,786
LOOP,871


Double-click __here__ for a hint

<!--
Order the previous query and limit the number of rows you fetch
-->

Double-click __here__ for the solution.

<!-- Solution:

%sql select Community_Area_Name, sum(College_Enrollment) AS TOTAL_ENROLLMENT \
   from SCHOOLS \
   group by Community_Area_Name \
   order by TOTAL_ENROLLMENT asc \
   fetch first 5 rows only

-->

### Problem 10

##### Get the hardship index for the community area which has College Enrollment of 4638

In [58]:
%sql select hardship_index from chicago_socioeconomic_data CSD, schools S\
    where CSD.ca = S.community_area_number and college_enrollment = 4368

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


hardship_index
6.0


Double-click __here__ for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, schools CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368

-->

### Problem 11

##### Get the hardship index for the community area which has the highest value for College Enrollment

In [67]:
%sql select CSD.ca, CSD.community_area_name,CSD.hardship_index from chicago_socioeconomic_data as CSD, schools as S\
    where CSD.ca = S.community_area_number and college_enrollment = (SELECT MAX(college_enrollment) FROM SCHOOLS)

 * ibm_db_sa://mjs55547:***@dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net:50000/BLUDB
Done.


ca,community_area_name,hardship_index
5.0,North Center,6.0


Double-click __here__ for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->

## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.

Copyright &copy; 2018 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).
